<a href="https://colab.research.google.com/github/wkd-woo/finance-statements/blob/main/%EB%88%84%EB%9D%BD%EC%B0%BE%EA%B8%B0V2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install -U finance-datareader
import pandas as pd
import numpy as np
import os
from tqdm import tqdm
import FinanceDataReader as fdr

In [4]:
통합재무정보 = pd.read_csv('/content/drive/MyDrive/22캡스톤디자인/재무일괄/일괄/statements.csv', index_col = 0,  dtype={'종목코드':'str'})

In [7]:
통합재무정보

,종목코드,사업연도,보고서종류,자산총계,자본총계,자본총계(비지배),부채총계,유동자산,당기순이익,당기순이익(비지배),매출액,현금및현금성자산,감가상각비,영업이익,매출총이익
0,060310,2021,3분기보고서,"65,694,406,823","39,604,250,285","346,899,422","26,090,156,538","28,475,453,922","2,141,910,533","168,854,904","18,062,116,780","6,020,012,116",0,"2,022,128,462","4,340,642,679"
1,060310,2021,1분기보고서,"57,532,354,433","37,025,096,733","160,552,033","20,507,257,700","19,475,037,181","-362,882,021","13,036,456","3,775,878,119","5,475,587,054",0,"-281,517,202","509,620,804"
2,060310,2021,반기보고서,"60,945,473,913","38,562,631,695","376,862,566","22,382,842,218","23,161,861,807","1,119,822,996","200,255,460","11,420,086,693","4,980,950,161",0,"1,056,624,172","2,578,306,966"
3,060310,2015,사업보고서,"67,359,971,696","37,939,808,456",0,"29,420,163,240","16,069,077,218","-5,599,032,927",0,"23,210,541,648","7,018,117,223",0,"-4,157,989,321","1,635,010,587"
4,060310,2016,3분기보고서,"59,740,105,216","35,061,777,226",0,"24,678,327,990","17,985,342,311","-2,606,151,383",0,"21,181,410,272","5,527,274,547",0,"-1,932,287,661","1,334,171,193"
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
38264,065650,2020,사업보고서,"47,217,129,660","27,856,442,820",0,"19,360,686,840","22,416,360,010","-3,521,169,069",0,"31,414,836,428","13,398,903,686",0,"-2,164,891,445","4,268,763,353"
38265,262840,2020,사업보고서,"29,801,051,915","20,070,268,768",0,"9,730,783,147","22,896,651,019","5,137,807,654",0,"15,270,479,176","903,083,315",0,"4,875,659,090","10,171,194,384"
38266,377190,2021,반기보고서,"757,741,121,157","325,909,527,744","14,329,183,694","431,831,593,413","21,431,619,143","2,150,044,521","154,836,078",0,"16,735,028,669",0,"4,865,784,111","9,201,108,120"
38267,334890,2021,반기보고서,"228,778,760,058","214,552,741,756",0,"14,226,018,302","44,761,246,058",0,0,"4,461,311,000","29,757,021,097",0,"4,160,253,012",0


# 데이터 가져옴

In [8]:
자산총계누락 = 통합재무정보[(통합재무정보['자산총계'] == '0')|(통합재무정보['자산총계'] == 0)]['종목코드'].unique().tolist()
부채총계누락 = 통합재무정보[(통합재무정보['부채총계'] == '0')|(통합재무정보['부채총계'] == 0)]['종목코드'].unique().tolist()
자본총계누락 = 통합재무정보[(통합재무정보['자본총계'] == '0')|(통합재무정보['자본총계'] == 0)]['종목코드'].unique().tolist()

매출액누락 = 통합재무정보[(통합재무정보['매출액'] == '0')|(통합재무정보['매출액'] == 0)]['종목코드'].unique().tolist()
영업이익누락 = 통합재무정보[(통합재무정보['영업이익'] == '0')|(통합재무정보['영업이익'] == 0)]['종목코드'].unique().tolist()
매출총이익누락 = 통합재무정보[(통합재무정보['매출총이익'] == '0')|(통합재무정보['매출총이익'] == 0)]['종목코드'].unique().tolist()
당기순이익누락 = 통합재무정보[(통합재무정보['당기순이익'] == '0')|(통합재무정보['당기순이익'] == 0)]['종목코드'].unique().tolist()


전체누락 = 자산총계누락 + 부채총계누락 + 자본총계누락 + 매출액누락 + 영업이익누락 + 매출총이익누락 + 당기순이익누락

In [9]:
print(f"자산총계누락: {len(자산총계누락)}")
print(f"부채총계누락: {len(부채총계누락)}")
print(f"자본총계누락: {len(자본총계누락)}")
print(f"매출액누락: {len(매출액누락)}")
print(f"영업이익누락: {len(영업이익누락)}")
print(f"매출총이익누락: {len(매출총이익누락)}")
print(f"당기순이익누락: {len(당기순이익누락)}")

print()
print('=========================')
print(f"전체누락: {len(전체누락)}")

자산총계누락: 15
부채총계누락: 3
자본총계누락: 5
매출액누락: 133
영업이익누락: 58
매출총이익누락: 184
당기순이익누락: 172

전체누락: 570


In [10]:
전체누락_unq = list(set(전체누락))
print(f"전체누락 unique: {len(전체누락_unq)}")

전체누락 unique: 381


In [11]:
import glob

재무상태표 = [] # _01_
손익계산서 = [] # _02_
포괄손익계산서 = [] # _03_
현금흐름표 = [] # _04_
자본변동표 = [] #_05_


for name in glob.glob('/content/drive/MyDrive/22캡스톤디자인/재무일괄/*.txt'):
  if '_01_' in name:
    재무상태표.append(name)
  elif '_02_' in name:
    손익계산서.append(name)
  elif '_03_' in name:
    포괄손익계산서.append(name)
  elif '_04_' in name:
    현금흐름표.append(name)
  elif '_05_' in name:
    자본변동표.append(name)

In [12]:
포괄손익계산샘플 = pd.DataFrame(columns=['종목코드','사업연도', '보고서종류', '회사명', '업종', '항목코드','항목명', '당기'])

for path in tqdm(포괄손익계산서):
  temp = pd.read_csv(path, delimiter = '\t', encoding='cp949')  

  당기 = ''
  if temp['보고서종류'][0] == '1분기보고서':
    당기 = '당기 1분기 누적' 
  elif temp['보고서종류'][0] == '반기보고서':
    당기 = '당기 반기 누적' 
  elif temp['보고서종류'][0] == '3분기보고서':
    당기 = '당기 3분기 누적' 
  elif temp['보고서종류'][0] == '사업보고서':
    당기 = '당기' 

  temp['사업연도'] = path[52:56] # 사업연도 컬럼 설정 - 파일명에서 가져온다
  temp['항목명'] = temp['항목명'].str.strip() # 항목명 컬럼 문자열 전처리
  temp['항목코드'] = temp['항목코드'].str.strip() # 항목코드 컬럼 문자열 전처리

  temp = temp[['종목코드','사업연도', '보고서종류', '회사명', '업종', '항목코드','항목명', 당기]]
  temp.rename(columns={당기:"당기"}, inplace=True)

  포괄손익계산샘플 = 포괄손익계산샘플.append(temp)

포괄손익계산샘플['종목코드'] = 포괄손익계산샘플['종목코드'].apply(lambda x:x[1:-1])

손익계산샘플 = pd.DataFrame(columns=['종목코드','사업연도', '보고서종류', '회사명', '업종', '항목코드','항목명', '당기'])

for path in tqdm(손익계산서):
  temp = pd.read_csv(path, delimiter = '\t', encoding='cp949')  

  당기 = ''
  if temp['보고서종류'][0] == '1분기보고서':
    당기 = '당기 1분기 누적' 
  elif temp['보고서종류'][0] == '반기보고서':
    당기 = '당기 반기 누적' 
  elif temp['보고서종류'][0] == '3분기보고서':
    당기 = '당기 3분기 누적' 
  elif temp['보고서종류'][0] == '사업보고서':
    당기 = '당기' 

  temp['사업연도'] = path[52:56] # 사업연도 컬럼 설정 - 파일명에서 가져온다
  temp['항목명'] = temp['항목명'].str.strip() # 항목명 컬럼 문자열 전처리
  temp['항목코드'] = temp['항목코드'].str.strip() # 항목코드 컬럼 문자열 전처리

  temp = temp[['종목코드','사업연도', '보고서종류', '회사명', '업종', '항목코드','항목명', 당기]]
  temp.rename(columns={당기:"당기"}, inplace=True)

  손익계산샘플 = 손익계산샘플.append(temp)

손익계산샘플['종목코드'] = 손익계산샘플['종목코드'].apply(lambda x:x[1:-1])

 21%|██        | 5/24 [00:16<00:41,  2.17s/it]/usr/local/lib/python3.7/dist-packages/pandas/core/frame.py:5047: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  errors=errors,
100%|██████████| 24/24 [00:01<00:00, 19.57it/s]


In [13]:
매출액누락_정보 = 통합재무정보[(통합재무정보['종목코드'].isin(매출액누락)*통합재무정보['매출액'] == '0')][['종목코드','사업연도','보고서종류','매출액']]
매출액누락_정보 = 매출액누락_정보.sort_values(by=['종목코드', '사업연도', '보고서종류'])
영업이익누락_정보 = 통합재무정보[(통합재무정보['종목코드'].isin(영업이익누락))&(통합재무정보['영업이익'] == '0')][['종목코드','사업연도','보고서종류','영업이익']]
영업이익누락_정보 = 영업이익누락_정보.sort_values(by=['종목코드', '사업연도', '보고서종류'])
매출총이익누락_정보 = 통합재무정보[(통합재무정보['종목코드'].isin(매출총이익누락))&(통합재무정보['매출총이익'] == '0')][['종목코드','사업연도','보고서종류','매출총이익']]
매출총이익누락_정보 = 매출총이익누락_정보.sort_values(by=['종목코드', '사업연도', '보고서종류'])
당기순이익누락_정보 = 통합재무정보[(통합재무정보['종목코드'].isin(당기순이익누락))&(통합재무정보['당기순이익'] == '0')][['종목코드','사업연도','보고서종류', '당기순이익']]
당기순이익누락_정보 = 당기순이익누락_정보.sort_values(by=['종목코드', '사업연도', '보고서종류'])

# 당기순이익 누락

In [14]:
df = pd.DataFrame(columns=['종목코드','사업연도', '보고서종류', '회사명', '업종', '항목코드','항목명', '당기'])
for row_tuple in tqdm(당기순이익누락_정보.iterrows()):
  row_종목코드 = row_tuple[1]['종목코드']
  row_사업연도 = row_tuple[1]['사업연도']
  row_보고서종류 = row_tuple[1]['보고서종류']

  df = df.append(손익계산샘플[(손익계산샘플['종목코드'] == row_종목코드)&(손익계산샘플['사업연도'] == str(row_사업연도))&(손익계산샘플['보고서종류'] == row_보고서종류)])
  df = df.append(포괄손익계산샘플[(포괄손익계산샘플['종목코드'] == row_종목코드)&(포괄손익계산샘플['사업연도'] == str(row_사업연도))&(포괄손익계산샘플['보고서종류'] == row_보고서종류)])

df

747it [02:58,  4.19it/s]


,종목코드,사업연도,보고서종류,회사명,업종,항목코드,항목명,당기
31442,000440,2017,반기보고서,중앙에너비스,467,ifrs_Revenue,매출액,"33,366,681,275"
31443,000440,2017,반기보고서,중앙에너비스,467,ifrs_CostOfSales,매출원가,"27,460,213,806"
31444,000440,2017,반기보고서,중앙에너비스,467,ifrs_GrossProfit,매출총이익,"5,906,467,469"
31445,000440,2017,반기보고서,중앙에너비스,467,dart_TotalSellingGeneralAdministrativeExpenses,판매비와관리비,"5,586,515,819"
31446,000440,2017,반기보고서,중앙에너비스,467,entity00149770_udf_IS_201781121644519_Statemen...,계속영업이익( 손실 ),"319,951,650"
...,...,...,...,...,...,...,...,...
1238,950170,2018,1분기보고서,JTC,471,ifrs_ComprehensiveIncome,총포괄손익,NaN
1239,950170,2018,1분기보고서,JTC,471,entity01041828_udf_IS_20187231720157_Comprehen...,지배기업의 소유주지분순이익,"486,129,255"
1240,950170,2018,1분기보고서,JTC,471,ifrs_EarningsPerShareAbstract,주당이익 [abstract],NaN
1241,950170,2018,1분기보고서,JTC,471,ifrs_BasicEarningsLossPerShare,기본주당이익(손실) (엔),14


In [15]:
df = df.replace(0, np.NaN)
df = df.replace('0', np.NaN)
df.dropna(axis=0, inplace=True)
df

,종목코드,사업연도,보고서종류,회사명,업종,항목코드,항목명,당기
31442,000440,2017,반기보고서,중앙에너비스,467,ifrs_Revenue,매출액,"33,366,681,275"
31443,000440,2017,반기보고서,중앙에너비스,467,ifrs_CostOfSales,매출원가,"27,460,213,806"
31444,000440,2017,반기보고서,중앙에너비스,467,ifrs_GrossProfit,매출총이익,"5,906,467,469"
31445,000440,2017,반기보고서,중앙에너비스,467,dart_TotalSellingGeneralAdministrativeExpenses,판매비와관리비,"5,586,515,819"
31446,000440,2017,반기보고서,중앙에너비스,467,entity00149770_udf_IS_201781121644519_Statemen...,계속영업이익( 손실 ),"319,951,650"
...,...,...,...,...,...,...,...,...
1236,950170,2018,1분기보고서,JTC,471,dart_OtherComprehensiveIncomeThatWillBeReclass...,당기손익으로 재분류될 수 있는 항목(세후기타포괄손익),"24,883,013"
1237,950170,2018,1분기보고서,JTC,471,ifrs_GainsLossesOnExchangeDifferencesOnTransla...,해외사업장환산외환차이(세후기타포괄손익),"24,883,013"
1239,950170,2018,1분기보고서,JTC,471,entity01041828_udf_IS_20187231720157_Comprehen...,지배기업의 소유주지분순이익,"486,129,255"
1241,950170,2018,1분기보고서,JTC,471,ifrs_BasicEarningsLossPerShare,기본주당이익(손실) (엔),14


In [16]:
df.to_csv('/content/drive/MyDrive/22캡스톤디자인/재무일괄/일괄/당기순이익누락추정.csv')

# 매출액누락

In [17]:
df2 = pd.DataFrame(columns=['종목코드','사업연도', '보고서종류', '회사명', '업종', '항목코드','항목명', '당기'])
for row_tuple in tqdm(매출액누락_정보.iterrows()):
  row_종목코드 = row_tuple[1]['종목코드']
  row_사업연도 = row_tuple[1]['사업연도']
  row_보고서종류 = row_tuple[1]['보고서종류']

  df2 = df2.append(손익계산샘플[(손익계산샘플['종목코드'] == row_종목코드)&(손익계산샘플['사업연도'] == str(row_사업연도))&(손익계산샘플['보고서종류'] == row_보고서종류)])
  df2 = df2.append(포괄손익계산샘플[(포괄손익계산샘플['종목코드'] == row_종목코드)&(포괄손익계산샘플['사업연도'] == str(row_사업연도))&(포괄손익계산샘플['보고서종류'] == row_보고서종류)])

df2

910it [04:46,  3.18it/s]


,종목코드,사업연도,보고서종류,회사명,업종,항목코드,항목명,당기
37637,000240,2016,1분기보고서,한국타이어월드와이드,715,entity00160047_udf_IS_2016515164639527_Stateme...,영업수익,"63,692,032,878"
37638,000240,2016,1분기보고서,한국타이어월드와이드,715,dart_TotalSellingGeneralAdministrativeExpenses,영업비용,"9,462,501,282"
37639,000240,2016,1분기보고서,한국타이어월드와이드,715,dart_OperatingIncomeLoss,영업이익(손실),"54,229,531,596"
37640,000240,2016,1분기보고서,한국타이어월드와이드,715,ifrs_FinanceIncome,금융수익,"2,592,826,093"
37641,000240,2016,1분기보고서,한국타이어월드와이드,715,ifrs_FinanceCosts,금융원가,"84,855,022"
...,...,...,...,...,...,...,...,...
7495,950220,2021,반기보고서,네오이뮨텍,701,ifrs-full_GainsLossesOnExchangeDifferencesOnTr...,해외사업환산손익,"-114,974"
7496,950220,2021,반기보고서,네오이뮨텍,701,ifrs-full_ComprehensiveIncome,총포괄손익,"-22,107,118"
7497,950220,2021,반기보고서,네오이뮨텍,701,ifrs-full_EarningsPerShareAbstract,주당이익 [abstract],NaN
7498,950220,2021,반기보고서,네오이뮨텍,701,ifrs-full_BasicEarningsLossPerShare,기본주당이익(손실),-1


In [18]:
df2 = df2.replace(0, np.NaN)
df2 = df2.replace('0', np.NaN)
df2.dropna(axis=0, inplace=True)
df2

,종목코드,사업연도,보고서종류,회사명,업종,항목코드,항목명,당기
37637,000240,2016,1분기보고서,한국타이어월드와이드,715,entity00160047_udf_IS_2016515164639527_Stateme...,영업수익,"63,692,032,878"
37638,000240,2016,1분기보고서,한국타이어월드와이드,715,dart_TotalSellingGeneralAdministrativeExpenses,영업비용,"9,462,501,282"
37639,000240,2016,1분기보고서,한국타이어월드와이드,715,dart_OperatingIncomeLoss,영업이익(손실),"54,229,531,596"
37640,000240,2016,1분기보고서,한국타이어월드와이드,715,ifrs_FinanceIncome,금융수익,"2,592,826,093"
37641,000240,2016,1분기보고서,한국타이어월드와이드,715,ifrs_FinanceCosts,금융원가,"84,855,022"
...,...,...,...,...,...,...,...,...
7493,950220,2021,반기보고서,네오이뮨텍,701,dart_OtherComprehensiveIncomeNetOfTaxGainsLoss...,순확정부채의 재측정손익(세후기타포괄손익),"3,584"
7495,950220,2021,반기보고서,네오이뮨텍,701,ifrs-full_GainsLossesOnExchangeDifferencesOnTr...,해외사업환산손익,"-114,974"
7496,950220,2021,반기보고서,네오이뮨텍,701,ifrs-full_ComprehensiveIncome,총포괄손익,"-22,107,118"
7498,950220,2021,반기보고서,네오이뮨텍,701,ifrs-full_BasicEarningsLossPerShare,기본주당이익(손실),-1


In [19]:
df2.to_csv('/content/drive/MyDrive/22캡스톤디자인/재무일괄/일괄/매출액누락추정.csv')

# 영업이익누락

In [20]:
df3 = pd.DataFrame(columns=['종목코드','사업연도', '보고서종류', '회사명', '업종', '항목코드','항목명', '당기'])
for row_tuple in tqdm(영업이익누락_정보.iterrows()):
  row_종목코드 = row_tuple[1]['종목코드']
  row_사업연도 = row_tuple[1]['사업연도']
  row_보고서종류 = row_tuple[1]['보고서종류']

  df3 = df3.append(손익계산샘플[(손익계산샘플['종목코드'] == row_종목코드)&(손익계산샘플['사업연도'] == str(row_사업연도))&(손익계산샘플['보고서종류'] == row_보고서종류)])
  df3 = df3.append(포괄손익계산샘플[(포괄손익계산샘플['종목코드'] == row_종목코드)&(포괄손익계산샘플['사업연도'] == str(row_사업연도))&(포괄손익계산샘플['보고서종류'] == row_보고서종류)])

df3

254it [00:59,  4.24it/s]


,종목코드,사업연도,보고서종류,회사명,업종,항목코드,항목명,당기
31442,000440,2017,반기보고서,중앙에너비스,467,ifrs_Revenue,매출액,"33,366,681,275"
31443,000440,2017,반기보고서,중앙에너비스,467,ifrs_CostOfSales,매출원가,"27,460,213,806"
31444,000440,2017,반기보고서,중앙에너비스,467,ifrs_GrossProfit,매출총이익,"5,906,467,469"
31445,000440,2017,반기보고서,중앙에너비스,467,dart_TotalSellingGeneralAdministrativeExpenses,판매비와관리비,"5,586,515,819"
31446,000440,2017,반기보고서,중앙에너비스,467,entity00149770_udf_IS_201781121644519_Statemen...,계속영업이익( 손실 ),"319,951,650"
...,...,...,...,...,...,...,...,...
30177,950140,2017,반기보고서,잉글우드랩,649,entity01165739_udf_IS_2017426122125948_udf_IS_...,매도가능금융자산평가손실의 대체,"2,518,898"
30178,950140,2017,반기보고서,잉글우드랩,649,entity01165739_udf_IS_2017426122141765_udf_IS_...,해외사업환산손익,"-323,658"
30179,950140,2017,반기보고서,잉글우드랩,649,ifrs_ComprehensiveIncome,총포괄이익,"-2,636,394"
30180,950140,2017,반기보고서,잉글우드랩,649,ifrs_EarningsPerShareAbstract,주당이익 [abstract],NaN


In [21]:
df3 = df3.replace(0, np.NaN)
df3 = df3.replace('0', np.NaN)
df3.dropna(axis=0, inplace=True)
df3

,종목코드,사업연도,보고서종류,회사명,업종,항목코드,항목명,당기
31442,000440,2017,반기보고서,중앙에너비스,467,ifrs_Revenue,매출액,"33,366,681,275"
31443,000440,2017,반기보고서,중앙에너비스,467,ifrs_CostOfSales,매출원가,"27,460,213,806"
31444,000440,2017,반기보고서,중앙에너비스,467,ifrs_GrossProfit,매출총이익,"5,906,467,469"
31445,000440,2017,반기보고서,중앙에너비스,467,dart_TotalSellingGeneralAdministrativeExpenses,판매비와관리비,"5,586,515,819"
31446,000440,2017,반기보고서,중앙에너비스,467,entity00149770_udf_IS_201781121644519_Statemen...,계속영업이익( 손실 ),"319,951,650"
...,...,...,...,...,...,...,...,...
30174,950140,2017,반기보고서,잉글우드랩,649,entity01165739_udf_IS_201782495255736_udf_IS_2...,순확정급여부채재측정요소,"-52,992"
30177,950140,2017,반기보고서,잉글우드랩,649,entity01165739_udf_IS_2017426122125948_udf_IS_...,매도가능금융자산평가손실의 대체,"2,518,898"
30178,950140,2017,반기보고서,잉글우드랩,649,entity01165739_udf_IS_2017426122141765_udf_IS_...,해외사업환산손익,"-323,658"
30179,950140,2017,반기보고서,잉글우드랩,649,ifrs_ComprehensiveIncome,총포괄이익,"-2,636,394"


In [22]:
df3.to_csv('/content/drive/MyDrive/22캡스톤디자인/재무일괄/일괄/영업이익누락추정.csv')

# 매출총이익누락

In [23]:
df4 = pd.DataFrame(columns=['종목코드','사업연도', '보고서종류', '회사명', '업종', '항목코드','항목명', '당기'])
for row_tuple in tqdm(매출총이익누락_정보.iterrows()):
  row_종목코드 = row_tuple[1]['종목코드']
  row_사업연도 = row_tuple[1]['사업연도']
  row_보고서종류 = row_tuple[1]['보고서종류']

  df4 = df4.append(손익계산샘플[(손익계산샘플['종목코드'] == row_종목코드)&(손익계산샘플['사업연도'] == str(row_사업연도))&(손익계산샘플['보고서종류'] == row_보고서종류)])
  df4 = df4.append(포괄손익계산샘플[(포괄손익계산샘플['종목코드'] == row_종목코드)&(포괄손익계산샘플['사업연도'] == str(row_사업연도))&(포괄손익계산샘플['보고서종류'] == row_보고서종류)])

df4

2180it [09:08,  3.97it/s]


,종목코드,사업연도,보고서종류,회사명,업종,항목코드,항목명,당기
29480,000230,2016,3분기보고서,일동홀딩스,212,ifrs_Revenue,영업수익,"33,158,126,975"
29481,000230,2016,3분기보고서,일동홀딩스,212,ifrs_CostOfSales,영업비용,"34,704,413,632"
29482,000230,2016,3분기보고서,일동홀딩스,212,dart_CostOfSalesFromSaleOfGoods,매출원가,"18,522,547,868"
29483,000230,2016,3분기보고서,일동홀딩스,212,entity00146083_udf_IS_2016119182524679_CostOfS...,판매비와관리비,"15,048,622,960"
29484,000230,2016,3분기보고서,일동홀딩스,212,entity00146083_udf_IS_2016119182527838_CostOfS...,연구개발비,"1,133,242,804"
...,...,...,...,...,...,...,...,...
7495,950220,2021,반기보고서,네오이뮨텍,701,ifrs-full_GainsLossesOnExchangeDifferencesOnTr...,해외사업환산손익,"-114,974"
7496,950220,2021,반기보고서,네오이뮨텍,701,ifrs-full_ComprehensiveIncome,총포괄손익,"-22,107,118"
7497,950220,2021,반기보고서,네오이뮨텍,701,ifrs-full_EarningsPerShareAbstract,주당이익 [abstract],NaN
7498,950220,2021,반기보고서,네오이뮨텍,701,ifrs-full_BasicEarningsLossPerShare,기본주당이익(손실),-1


In [24]:
df4 = df4.replace(0, np.NaN)
df4 = df4.replace('0', np.NaN)
df4.dropna(axis=0, inplace=True)
df4

,종목코드,사업연도,보고서종류,회사명,업종,항목코드,항목명,당기
29480,000230,2016,3분기보고서,일동홀딩스,212,ifrs_Revenue,영업수익,"33,158,126,975"
29481,000230,2016,3분기보고서,일동홀딩스,212,ifrs_CostOfSales,영업비용,"34,704,413,632"
29482,000230,2016,3분기보고서,일동홀딩스,212,dart_CostOfSalesFromSaleOfGoods,매출원가,"18,522,547,868"
29483,000230,2016,3분기보고서,일동홀딩스,212,entity00146083_udf_IS_2016119182524679_CostOfS...,판매비와관리비,"15,048,622,960"
29484,000230,2016,3분기보고서,일동홀딩스,212,entity00146083_udf_IS_2016119182527838_CostOfS...,연구개발비,"1,133,242,804"
...,...,...,...,...,...,...,...,...
7493,950220,2021,반기보고서,네오이뮨텍,701,dart_OtherComprehensiveIncomeNetOfTaxGainsLoss...,순확정부채의 재측정손익(세후기타포괄손익),"3,584"
7495,950220,2021,반기보고서,네오이뮨텍,701,ifrs-full_GainsLossesOnExchangeDifferencesOnTr...,해외사업환산손익,"-114,974"
7496,950220,2021,반기보고서,네오이뮨텍,701,ifrs-full_ComprehensiveIncome,총포괄손익,"-22,107,118"
7498,950220,2021,반기보고서,네오이뮨텍,701,ifrs-full_BasicEarningsLossPerShare,기본주당이익(손실),-1


In [25]:
df4.to_csv('/content/drive/MyDrive/22캡스톤디자인/재무일괄/일괄/매출총이익누락추정.csv')

---



```javascript
function ClickConnect(){
    console.log("코랩 연결 끊김 방지"); 
    document.querySelector("colab-toolbar-button#connect").click() 
}
setInterval(ClickConnect, 60 * 1000)

```

